### Exploring Datasets with pandas

Dataset tersebut berisi data tahunan arus imigran internasional yang dicatat oleh negara tujuan. Data tersebut menyajikan baik arus masuk maupun arus keluar menurut tempat lahir, kewarganegaraan atau tempat tinggal sebelumnya / selanjutnya baik bagi asing maupun warga negara. Versi saat ini menyajikan data yang berkaitan dengan 45 negara.

Di sesi ini, kita akan fokus pada data imigrasi Kanada.

pandas Basics
Sembari kita merivew yang telah kita pelajari di minggu lalu, kita akan melakukan beberapa hal dengan Pandas.

Hal pertama yang akan kita lakukan adalah mengimpor dua modul analisis data utama: Pandas dan Numpy.


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pip install openpyxl

Kita coba mendownload atau melihat file xls tersebut yang akan kita ambil adalah pada sheet ke dua dengan nama "Canada by Citizenship" 
 Membersihkan data beberapa kolom yang rangkap 
 Merename beberapa kolom

In [ ]:
df_can= pd.read_excel('https://github.com/ardhiraka/PFDS_sources/blob/master/Canada.xlsx?raw=true',
                       sheet_name='Canada by Citizenship',
                       skiprows=range(20),
                       skipfooter=2) #engine='openpyxl'
print("Data read into a pandas dataframe!")

In [ ]:
df_can.head()

In [ ]:
df_can.tail()

Saat menganalisa kumpulan data, selalu merupakan ide yang baik untuk memulai dengan mendapatkan dasr tentang kerangka data kita dengan menggunakan info()

In [ ]:
df_can.info()

In [ ]:
df_can.dropna(axis=1, how='all', inplace=True)

In [ ]:
df_can.columns

In [ ]:
df_can.loc[:,1980:2013] = df_can.loc[:,1980:2013].astype('Int64')

untuk mendapatkan list header kolom kita dapat memanggil parameter .columns daraframe

In [ ]:
df_can.columns.values

Demikian pula untuk mendapatkan daftar indeks kita menggunakan paramaeter .index

In [ ]:
df_can.index.values

untuk melihat dimensi dataframe, kita menggunakan paramater .shape

In [ ]:
# size of dataframe (rows, columns)
df_can.shape

Mari bersihkan dataset dengan menghapus beberapa kolom yang tidak perlu. Kita bisa menggunakan metode pandas drop() sebagai berikut

In [ ]:
# in pandas axis=0 represents rows (default) and axis=1 represents columns.
df_can.drop(['AREA', 'REG','DEV','Type', 'Coverage'], axis=1, inplace=True)
df_can.head(2)

In [ ]:
df_can.dropna(axis=0, inplace=True)
df_can.tail(2)

Mari ganti nama kolom agar make sense. Kita bisa menggunakan metode rename() dengan memasukkan dictionary nama lama dan baru sebagai berikut:

In [ ]:
df_can.rename(columns={'OdName':'Country', 'AreaName':'Continent', 'RegName':'Region'}, inplace=True)
df_can.columns

Kita juga akan menambahkan kolom 'Total' yang merangkum total imigran menurut negara selama periode 1980 - 2013, sebagai berikut:

In [ ]:
df_can['Total'] = df_can.sum(axis=1)


In [ ]:
df_can['Total'] = df_can['Total'].astype('Int64')

Memeriksa untuk melihat berapa banyak objke null yang kita miliki dalam dataset sebagai berikut

In [ ]:
df_can.isnull().sum()

In [ ]:
print(df_can.groupby('Country').sum())

Terakhir, mari kita lihat ringkasan singkat dari setiap kolom dalam kerangka data kita menggunakan metode describe().

In [ ]:
df_can.describe()

### pandas: Indexing and Selection (slicing)

Metode 1: Cepat dan mudah, tetapi hanya berfungsi jika nama kolom TIDAK memiliki spasi atau karakter khusus.

Mari coba memfilter daftar negara dan data selama tahun 1980 - 1985

In [ ]:
df_can[['Country', 1980,1981,1982,1983,1984,1985]] # return a dataframe

Select Row

Ada 3 cara utama untuk memilih baris:

df.loc[label]        
        #filters by the labels of the index/column
df.iloc[index]       
        #filters by the positions of the index/column

Sebelum kita melanjutkan, perhatikan bahwa indeks default dari dataset adalah rentang numerik dari 0 hingga 194. Hal ini membuatnya sangat sulit untuk melakukan kueri berdasarkan negara tertentu. Misalnya untuk mencari data di Jepang, kita perlu mengetahui nilai indeks yang sesuai.

Ini dapat diperbaiki dengan sangat mudah dengan menyetel kolom 'Country' sebagai indeks menggunakan metode set_index().

In [ ]:
df_can.set_index('Country', inplace=True)

In [ ]:
df_can.head(3)

In [ ]:
# optional: to remove the name of the index
df_can.index.name = None

Contoh: Mari kita lihat jumlah imigran dari Jepang (baris 87) untuk skenario berikut ini: 1. Data baris lengkap (semua kolom) 2. Untuk tahun 2013 3. Untuk tahun 1980 hingga 1985


In [ ]:
# 1. the full row data (all columns)
japan = df_can.loc['Japan']
print(japan)

# alternate methods
print(df_can.iloc[87])

In [ ]:
print(df_can[df_can.index == 'Japan'].T.squeeze())

In [ ]:
# for year 2013
print(df_can.loc['Japan', 2013])

#alternate method
print(df_can.iloc[87, 36])

In [ ]:
#3. for years 1980 to 1985
print(df_can.loc['Japan', [1980,1981,1982,1983,1984,1985]]) 

# alternate method
print(df_can.iloc[87, [3,4,5,6,7,8]])

Nama kolom yang merupakan bilangan bulat (seperti tahun) mungkin menimbulkan kebingungan. Misalnya, ketika kita mengacu pada tahun 2013, orang mungkin bingung ketika 2013 adalah indeks posisi 2013.

Untuk menghindari ambuigitas ini, mari kita ubah nama kolom menjadi string: '1980' hingga '2013'.

In [ ]:
# Karena kita mengonversi tahun menjadi string, 
# mari kita deklarasikan variabel yang akan memungkinkan kita untuk dengan mudah memanggil rentang tahun penuh:
# useful for plotting later on
years = list(map(str, range(1980, 2014)))
years

In [ ]:
df_can.columns = list(map(str, df_can.columns))

### Filtering based on a criteria

Untuk memfilter dataframe berdasarkan suatu kondisi, kita cukup meneruskan kondisi tersebut sebagai vektor boolean.

Misalnya, mari kita filter kerangka data untuk menampilkan data di negara-negara Asia (AreaName = Asia).

In [ ]:
# 1. create teh condition boolean series
condition = df_can['Continent'] == 'Asia'
print(condition)

In [ ]:
df_can[condition]

In [ ]:
# useful for plotting later on
years = list(map(str, range(1980, 2014)))
years

In [ ]:
# we can pass mutliple criteria in the same line. 
# let's filter for AreaNAme = Asia and RegName = Southern Asia

df_can[(df_can['Continent']=='Asia') & (df_can['Region']=='Southern Asia')]

# note: When using 'and' and 'or' operators, pandas requires we use '&' and '|' instead of 'and' and 'or'
# don't forget to enclose the two conditions in parentheses

Sebelum kita melanjutkan: mari kita tinjau perubahan yang telah kita buat pada kerangka data kita.

In [ ]:
print('data dimensions:', df_can.shape)
print(df_can.columns)
df_can.head(2)

### Visualizing Data using Matplotlib

Definisi

Visualisasi data dilihat oleh berbagai bidang ilmu sebagai salah satu alat komunikasi modern. 

Tujuan utama dari visualisasi data adalah untuk mengkomunikasikan informasi secara jelas dan efisien kepada penggunanya melalui grafik infomasi yang dipilih, seperti tabel tabel atau grafik. 

Visualisasi yang efektif membantu penggunanya dalam menganalisa dan penalaran tentang data dan bukti, sehingga data yang kompleks menjadi mudah dipahami dan berguna.

Data Visualisasi adalah teknik yang digunakan untuk mengkomunikasikan data atau informasi dengan membuatnya kedalam objek visual (contohnya, titik, garis, batang, dan lainnya) dalam grafik.

Ada dua jenis visualisasi data yang paling populer adalah dashboard dan infografis, keduanya menggunakan kombinasi grafik, teks, dan gambar untuk mengkomunikasikan pesan data.

dalam membuat visualisasi yaitu Who, What, and How. Ketika anda melakukan explanatory analysis ada beberapa hal tadi yang harus diperhatikan. 
1. Siapa audience nya ?  agar visualisasi yang ditampilkan menjadi relevan. 
2. Apa yang ingin kita beritahu ke audience ? 
3. Bagaimana kita menjelaskan dalam pengunaan data

Poin Penting Dalam Visualisasi Data

Visualisasi data yang bagus memiliki kriteria sebagai berikut :
Menarik secara visual; Dengan menggunakan desain visualisasi data yang menarik, akan meningkatkan ketertarikan para pengguna dalam mengkomunikasikan informasi tersebut.


Scalable; Visualisasi data yang baik adalah yang mampu dalam hal aksesibilitas dan dapat dipelihara serta dimodifikasi di masa depan.
Memberi pengguna informasi yang benar; Dengan menentukan suatu tepat kebutuhan para penggunanya dan bagaimana cara penggunaanya.
Dapat diakses; Visualisasi data harus dapat diakses pada perangkat apa pun, kapan saja, dan dimana saja.
Memungkinkan pengembangan dan penyebaran yang cepat; Pengguna membutuhkan informasi dalam jangka waktu dan kepentingan tertentu, maka adanya kemungkinan ada nya waktu tertentu dalam segi pengembangan visualisasi data dan penyebaran dari hasil visualisasi data.

### Matplotlib.pyplot

Salah satu aspek inti Matplotlib adalah matplotlib.pyplot. matplotlib.pyplot adalah kumpulan fungsi gaya perintah yang membuat Matplotlib berfungsi seperti MATLAB. Setiap fungsi pyplot membuat beberapa perubahan pada gambar: misalnya, membuat gambar, membuat area plot dalam gambar, membuat plot beberapa garis di area plotting, menghiasi plot dengan label, dll. Di sesi ini, kita akan bekerja dengan scripting layer untuk mempelajari cara membuat plot garis.

In [ ]:
pip install --upgrade pip

In [ ]:
pip install matplotlib

In [ ]:
# we are using the inline backend
# magic command Notebook jupyter dan jupyter Qtconsole

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
print ('Matplotlib Version', mpl.__version__)

In [ ]:
print(plt.style.available)
mpl.style.use(['ggplot'])

### Membuat DIagram Garis

Diagram garis dapat dibuat dengan function plot()  , penggunaannya plot(x,y) , x dan y adalah pasangan data 

In [ ]:
plt.plot([2,3,4,5],[20,25,16,11])  # data untuk x dan y ini bisa berupa list, numpy array dan tuple
plt.show()

### Membuat Title dan label dari sumbu x dan y

In [ ]:
# kita akan menambahkam judul di dalam axes / grafik plot
plt.plot([2,3,4,5],[20,25,16,11]) # data x dan y bisa berupa list, numpy array dan tuple
plt.title('Judul Plot')
plt.xlabel("Sumbu x")
plt.ylabel("sumbu y")
plt.show()

### Menyiapkan Figure dengan parameter figsize dalam satuan inchi

In [ ]:
# Kita dapat mengatur ukuran dari figure dengan function figure() dengan menggunakan setingan faram pada figsize
plt.figure(figsize=(7,4)) # satuan inch (x, y)
plt.plot([2,3,4,5],[20,25,16,11])
plt.title("Judul Plot")
plt.xlabel("Sumbu x")
plt.ylabel("Sumbu y")
plt.show()

### Garis, Default garis tidak terputus "-" dengan menggabungkan warna

In [ ]:
plt.figure(figsize=(7,4))
plt.plot([2,3,4,5],[20,25,16,11], "b-")
plt.title("Judul Plot")
plt.xlabel("Sumbu x")
plt.ylabel("Sumbu y")
plt.show()

### Garis, Default garis terputus "--" dengan menggabungkan warna "g--"

In [ ]:
# secara default garis tidak terputus 
plt.figure(figsize=(7,4))  # satuannya inch (x,y) 
plt.plot([2,3,4,5],[20,25,16,11],"g--")  # terputus 
plt.title("Judul Plot")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu y")
plt.show()

### Grafik berupa titik/dot, dengan menggabungkan warna

In [ ]:
# selain membuat garis pada plot bisa juga membuat grafik plot berupa titik 
plt.figure(figsize=(7,4))
plt.plot([2,3,4,5],[20,25,16,11], "bo")
plt.title("Judul Plot")
plt.xlabel("Sumbu x")
plt.ylabel("Sumbu y")
plt.show()

### Graafik berupa tanda x, dengan menggabungkan warna

In [ ]:
plt.figure(figsize=(7,4))  # satuannya inch (x,y) 
plt.plot([2,3,4,5],[20,25,16,11],"bx")  # titik x 
plt.title("Judul Plot")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu y")
plt.show()

### Grafik 2 buah bentuk dalam saru axes / grafik plot

In [ ]:
# megabungkan 2 buah bentuk dalam satu axes / grafik plot 
# bentuk garis dan dot

x = [2,3,4,5]
y = [20,25,16,11]
x = [2,3,4,5]
y = [20,25,16,11]
plt.plot(x,y,"r--",x,y,"bo")  # titik x 
plt.title("Judul Plot")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu y")
plt.show()

### Grafik 2 buah bentuk dalam satu axes / grafik plot antara titik dan x

In [ ]:
import numpy as np
x = np.array([1,2,3,4,5,6])
y0 = 2 * x
y1 = 2 ** x

plt.plot(x,y0, "rx", x,y1, "bo")
plt.title("Judul Plot")
plt.xlabel("Sumbu x")
plt.ylabel("Sumbu y")
plt.show()

### Keterangan Grafik dengan legend

In [ ]:
# untuk memudahkan dalam membaca grafik kita menambahkan func legend = memberikan keterangan grafik plt 
x = np.array([1,2,3,4,5,6])
y0 = 2 * x
y1 = 2 ** x
plt.plot(x,y0,"r-", x,y1,"b--")
plt.title("Judul Plot")
plt.xlabel("Sumbu x")
plt.ylabel("Sumbu y")
plt.legend(['y0 = 2x', 'y1 = 2 ^ x'], loc='best') # lokasi yang terbaik
plt.show()

beberapa lokasi legend = "upper right" , "upper left" , "lower left" , "right" , 
"center left" , "center right" , "lower center" , "uper center" , "center"

### Membuat Diagram Batang 

Diagram batang atau diagram bar chart itu berbentuk persegi panjang 
biasanya di pakai untuk mengambarkan perkembangan nilai dari suatu objek 
diagram batang juga menampilkan beberapa description secara vertikal atau horizontal 

function yang digunakan bisa bar() dan barh() 
bar() untuk vertikal 
barh() untuk horizontal 

In [ ]:
kelas = ['AA','BB','CC','DD','EE','FF']
jumlah_mahasiswa = [20,30,15,40,45,50]
plt.bar(kelas,jumlah_mahasiswa, color="red")
plt.title("Jumlah Mahasiswa Kampus XYZ")
plt.xlabel('Kelas')
plt.ylabel('Jumlah')
plt.show()

In [ ]:
kelas = ['AA','BB','CC','DD','EE','FF']
jumlah_mahasiswa = [20,30,15,40,45,50]
plt.barh(kelas,jumlah_mahasiswa, color='red') # horizontal
plt.title("Jumlah Mahasiswa Kampus XYZ")
plt.xlabel("Kelas")
plt.ylabel("Jumlah") 
plt.show()

### Membuat Diagram Pie

Diagram pie atau diagram pie chart meyajikan data menggunakan lingkaran 

Data yang ditampilkan biasanya dalam persentase data yang menginformasikan proporsi obyek suatu data terhadap keseluruhan data 

Jadi yang perlu kita ketahui itu besaran persentase data dan besaran sudut pusat sektor lingkaran 

untuk mengambarkannya menggunakan function pie()  , 

function pie() tidak mempunyai sumbu , parameter hanya satu x adalah nilai data yang diketahui pie(x) . 

Beberapa parameter  :

* autopct  -> untuk menampilkan nilai persentase 
* label  -> menampilkan keterangan label pada setiap object 
* startangle  - > memutar posisi lingkaran 

In [ ]:
# plt.pie?
posisi = ["Developer", "IT SUpport", "Network Engineer", "DevOps","DBA", "Data Science", " Data Engineer", "QA Engineer"]
jumlah = [40,5,5,7,4,10,5,15]
plt.pie(jumlah, autopct="%1.0f%%", labels=posisi, startangle=60) # 60 derajat
plt.axis('equal')
plt.title("Departement Technology")
plt.show

### Membuat Histogram

Histogram merupakan grafik yang menampilkan frekuensi data dalam bentuk diagram batang , 

sering digunakan dalam statistik atau probabilitas serta dasar dari berbagai macam distribusi statistik :  distribusi normal,  distribusi-t dll

In [ ]:
data = np.random.randn(1000)
plt.hist(data,10)  # jumlah kelompok kelas data yang diinginkan 
plt.show()
#menampilkan seperti kurva distibusi normal


### Membuat Scatter Plot

Scatter plot , scatter chart atau scatter diagram digunakan melihat sebaran data antara dua variabel 

Dengan melihat sebaran data , kita dapat mengetahui hubungan atau korelasi antara dua variabel tersebut 

Bisa menjadi korelasi positif negatif atau tidak ada korelasinya 

function scatter()  dengan scatter(x,y ) 

In [ ]:
tinggi =[163,178,170,168,170,162,164,175]
berat = [81,85,76,65,85,75,80,69]
plt.scatter(berat,tinggi)
plt.xlim(60,90) # batas minimum dan maksimum berat
plt.ylim(160,180) # batas minumum dan maksimum tinggi
plt.xlabel("Berat")
plt.ylabel("Tinggi")
plt.show()

### Menggabungkan beberapa grafik plot, 1 Figure

dalam 1 figure kita bisa membuat beberapa grafik plot digunakan function subplot (m,n,i) 
subplot(m,n,i) 
parmeter m dan n  : sebagai dimensi dari grafik plot 
parameter i : sebagai urutuan dari grafik plot 

Contoh : 
4 Grafik secara vertikal 
subplot(4,1,i)  akan ada 4 grafik dalam 4 kolom 
4 Graifk secara horizonal 2 baris 
subplot(2,2,i) akan ada 2 grafik di baris 1 dan 2 grafik di bawahnya

2 Graifk horizontal
subplot(1,2,i)  akan ada 2 grafik dalam 1 baris 

In [ ]:

x = np.array([1,2,3,4,5,6])
y0 = x ** 2
y1 = x ** 3

plt.subplot(1,2,1)
plt.plot(x,y0, "r-")
plt.title("Judul plot Y0")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu Y")
plt.legend(["Y0 = x ^ 2"])

plt.subplot(1,2,2)
plt.plot(x,y1,"r--")
plt.title("Judul plot Y1")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu Y")
plt.legend(["Y1 = x ^ 3"])

plt.suptitle("2 Grafik plot")
plt.show()


In [ ]:
x = np.array([1,2,3,4,5,6])
y0 = x ** 2
y1 = x ** 3

plt.subplot(2,1,1)
plt.plot(x,y0,"r-")
plt.title("Judul plot Y0")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu y")
plt.legend(["Y0 = x ^ 2"])

plt.subplot(2,1,2)
plt.plot(x,y1,"r--")
plt.title("Judul plot Y1")
plt.xlabel("Sumbu X")
plt.ylabel("Sumbu Y")
plt.legend(["Y1 = x ^ 3"])

plt.suptitle("2 Grafik plot")
plt.show()

### Plotting in pandas

Untungnya, pandas memiliki implementasi Matplotlib bawaan yang dapat kita gunakan. Membuat plot di pandas semudah menambahkan metode .plot() ke series atau dataframe.

Documentation:

Plotting with Series
Plotting with Dataframes

### Line Pots (Series/Dataframe)

What is a line plot and why use it?

Bagan garis atau line plot adalah jenis plot yang menampilkan informasi sebagai rangkaian titik data yang disebut 'markers' yang dihubungkan oleh segmen garis lurus. Line Plot adalah tipe bagan dasar yang umum di banyak bidang. Gunakan line plot jika kalian memiliki kumpulan data continuous. Ini paling cocok untuk visualisasi data berbasis tren selama periode waktu tertentu.

Let's start with a case study:

Pada tahun 2010, Haiti mengalami bencana gempa bumi berkekuatan 7,0. Gempa tersebut menyebabkan kerusakan yang luas dan korban jiwa dan sekitar tiga juta orang terkena dampak bencana alam ini. Sebagai bagian dari upaya kemanusiaan Kanada, Pemerintah Kanada meningkatkan upayanya dalam menerima pengungsi dari Haiti. Kita dapat dengan cepat memvisualisasikan upaya ini menggunakan Line plot:

Question: Plot a line graph of immigration from Haiti using df.plot().

Pertama, kita akan mengekstrak series data untuk Haiti.

### Kembali ke data awal df_can

In [ ]:
haiti = df_can.loc['Haiti', years]  #passing in years 1980 -2013 to exclude the 'total' column
haiti.head()

In [ ]:
haiti.index

In [ ]:
haiti.values

In [ ]:
print(haiti)

Selanjutnya, kita akan memplot plot garis dengan menambahkan .plot() ke haiti dataframe.

In [ ]:
haiti.plot()

pandas secara otomatis mengisi sumbu x dengan nilai indeks (years), dan sumbu y dengan nilai kolom (population). Namun, perhatikan bagaimana tahun tidak ditampilkan karena merupakan tipe string. Oleh karena itu, mari kita ubah jenis nilai indeks menjadi integer untuk plot.

Juga, mari beri label sumbu x dan y menggunakan plt.title(), plt.ylabel(), dan plt.xlabel() sebagai berikut:

In [ ]:
haiti.index = haiti.index.map(int) #let's change the index values of Haiti to type integer for plotting
haiti.plot(kind='line')

plt.title('Imigration from Haiti')
plt.xlabel('Years')
plt.ylabel('Number of imigrations')

plt.show()

In [ ]:
haiti.plot(kind='line')

plt.title('Immigration from Haiti')
plt.ylabel('Number of Immigrants')
plt.xlabel('Years')

# annotate the 2010 Earthquake. 
# syntax: plt.text(x, y, label)
plt.text(2000, 6000, '2010 Earthquake') # see note below

plt.show() 

Hanya dengan beberapa baris kode, kalian dapat dengan cepat mengidentifikasi dan memvisualisasikan lonjakan imigrasi!

Catatan singkat tentang nilai x dan y di plt.text(x, y, label):

Karena sumbu x (tahun) adalah tipe 'bilangan bulat', kami menetapkan x sebagai tahun. 

Sumbu y (jumlah pendatang) bertipe 'bilangan bulat', jadi kita tinggal menentukan nilai y = 6000.

    plt.text(2000, 6000, '2010 Earthquake') # years stored as type int

Jika tahun disimpan sebagai tipe 'string', kita perlu menentukan x sebagai posisi indeks tahun tersebut. Misalnya indeks ke-20 adalah tahun 2000 karena merupakan tahun ke-20 dengan tahun dasar 1980.

    plt.text(20, 6000, '2010 Earthquake') # years stored as type int


We will cover advanced annotation methods in later session.

Kita dapat dengan mudah menambahkan lebih banyak negara ke plot garis untuk membuat perbandingan imigrasi dari berbagai negara.

Pertanyaan: Mari kita bandingkan jumlah imigran dari India dan China dari tahun 1980 hingga 2013.

Step 1: Dapatkan kumpulan data untuk China dan India, dan tampilkan dataframe.


In [ ]:
China_India = df_can.loc[['China', 'India'], years]
China_India

Step 2: Plotting grafiknya. Kita akan secara eksplisit membuat line plot dengan meneruskan parameter kind ke plot().

In [ ]:
# type tour answer here
China_India.plot(kind='line')

Kelihatannya tidak benar...

Ingatlah bahwa pandas memplot indeks pada sumbu x dan kolom sebagai garis individu pada sumbu y. Karena df_CI adalah dataframe dengan country sebagai indeks dan years sebagai kolom, pertama-tama kita harus mengubah susunan kerangka data menggunakan metode transpose() untuk menukar baris dan kolom.

In [ ]:
df_CI = China_India.transpose()
df_CI.head()

pandas secara otomatis akan membuat grafik kedua negara pada grafik yang sama. Lanjutkan dan plot kerangka data baru yang sudah ditranspose. Pastikan untuk menambahkan judul ke plot dan memberi label pada sumbu.

In [ ]:
df_CI.plot(kind='line')
plt.title('Imigrants from China and India')
plt.ylabel('Number of Imigrants')
plt.xlabel('Years')
plt.show()

Dari plot di atas, kita dapat mengamati bahwa China dan India memiliki tren imigrasi yang sangat mirip selama bertahun-tahun.

Line plot adalah alat praktis untuk menampilkan beberapa variabel dependen terhadap satu variabel independen. Namun, disarankan tidak lebih dari 5-10 garis pada satu grafik; lebih dari itu dan menjadi sulit untuk ditafsirkan.

Question: Compare the trend of top 5 countries that contributed the most to immigration to Canada.




In [ ]:
df_can.sort_values(by='Total', ascending=False, axis=0, inplace=True)


In [ ]:
df_top5 = df_can.head(5)

In [ ]:
df_top5 = df_top5[years].transpose()

In [ ]:
df_top5.index = df_top5.index.map(int)

In [ ]:
df_top5.plot(kind='line', figsize=(14,8))
plt.title('Immigration Trend of Top 5 COuntries')
plt.ylabel('Number of Immigrants')
plt.xlabel('Years')
plt.show()

### Area Plots

Kita telah membuat plot garis yang mempisualisasikan 5 negara teratas yang menyummbangkan paling banyak imigran ke kanada dari 1980 hinga 2013. Dengan sedikit modifikasi pda kode, kita dapat mempisualisasikan pot ini sebagai xommulative plot, juga di kenala sebagai Stacked Line Plot atau Aarea plot

In [ ]:
df_can.sort_values(['Total'], ascending=False, axis=0, inplace=True)
#get the top 5 entries
df_top5 = df_can.head()

#transpose the dataframe
df_top5 = df_top5[years].transpose()

df_top5.head()

Area plot ter-stacked secara default, untuk menghasilakn satacked area polt, setiap kolom harus semua berupa nilai positif atau semua nilai negatif(nilai NaN akan di defaultkan secara default ke 0)
untuk menghasilkan unstacked plot, berikan 'stacked=False'

In [ ]:
df_top5.index = df_top5.index.map(int) # let's change the index values of df_top5 to type integer for plotting

df_top5.plot(kind='area',
             stacked=False,
             figsize=(20,10)) # pass a tuple (x,y) size

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

plot unstacked memiliki transparansi default(alpha value) sebesar 0,5. Kita dapat mengubah nilai ini dengan memasukan parameter alpha

In [ ]:
df_top5.plot(kind='area',
             alpha=0.25,
             stacked=False,
             figsize=(20,10)) # pass a tuple (x,y) size

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

### Two types of plotting

Ada dua gaya / pilihan membuat plot dengman matplotlib. Membuat plot menggunakan Artist layer dan membuat plot menggunakan scripting layer.

**Option 1: Scripting layer (procedural method) - using matplotlib.pyplot as 'plt' **

Kita dapat menggunakan plt yaitu matmplotlib.pyplot dan menambahkan lebih banyak elemen dengan memanggil metode yang berbeda secara prosedural; 

misalnya, plt.title(...) untuk menambahkan judul atau plt.xlabel(...) untuk menambahkan label ke sumbu x.

In [ ]:
# Option 1: This is what we have been using so far
df_top5.plot(kind='area',alpha=0.35,figsize=(20,10)) # pass a tuple (x,y) size

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

**Option 2: Artist layer (Object oriented method) - using an Axes instance from Matplotlib (preferred) **

Kita dapat menggunakan instance Axes dari plot kita saat ini dan menyimpannya dalam variabel (misalnya ax).

Kita dapat menambahkan lebih banyak elemen dengan memanggil metode dengan sedikit perubahan dalam sintaks (dengan menambahkan "set_" ke metode sebelumnya). 

Misalnya, gunakan ax.set_title() sebagai ganti plt.title() untuk menambahkan judul, atau ax.set_xlabel() sebagai ganti plt.xlabel() untuk menambahkan label ke sumbu x.

Opsi ini terkadang lebih transparan dan fleksibel untuk digunakan pada plot tingkat lanjut (khususnya jika memiliki banyak plot, seperti yang akan kita lihat nanti).

Di sesi ini, kita akan tetap menggunakan scripting layer, kecuali untuk beberapa visualisasi lanjutan di mana kita perlu menggunakan artist layer untuk memanipulasi aspek lanjutan dari plot.

In [ ]:
# option 2: preferd option with more flexibility
df_top5.plot(kind='area',alpha=0.35,figsize=(20,10)) # pass a tuple (x,y) size

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

Pertanyaan: Gunakan lapisan skrip untuk membuat plot area bertumpuk dari 5 negara yang berkontribusi paling sedikit terhadap imigrasi ke Kanada dari tahun 1980 hingga 2013. Gunakan nilai transparansi 0,45.

In [ ]:
# type your answer here
df_least5 = df_can.tail(5)
df_least5 = df_least5[years].transpose()

df_least5.plot(kind='area',
               alpha = 0.45,
               stacked=True,
               figsize =(20,10))

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

Pertanyaan: Gunakan layer artis untuk membuat plot area yang tidak ditumpuk dari 5 negara yang berkontribusi paling sedikit terhadap imigrasi ke Kanada dari tahun 1980 hingga 2013. Gunakan nilai transparansi 0,55.

In [ ]:
ax = df_least5.plot(kind='area',alpha = 0.45,figsize =(20,10))

plt.title('Immigration Trend of Top 5 Countries')
plt.ylabel('Number of Immigration')
plt.xlabel('Years')
plt.show()

### Histogram

Histogram adalah cara untuk merepresentasikan distribusi frekuensi dari dataset numerik. 

Cara kerjanya adalah mempartisi sumbu x menjadi beberapa bin, 

menetapkan setiap titik data dalam kumpulan data kita ke sebuah bin, 

dan kemudian menghitung jumlah titik data yang telah ditetapkan ke setiap bin. 

Jadi sumbu y adalah frekuensi atau banyaknya titik data pada setiap bin. 

Note bahwa kita dapat mengubah ukuran bin dan biasanya kita perlu menyesuaikannya agar distribusinya ditampilkan dengan baik.

Pertanyaan: Bagaimana distribusi frekuensi jumlah (penduduk) imigran baru dari berbagai negara ke Kanada pada tahun 2013?

Sebelum kita melanjutkan dengan membuat plot histogram, 

pertama-tama mari kita periksa data yang akan dibagi menjadi beberapa interval. 

Untuk melakukan ini, kami akan menggunakan metode Numpy's histrogram untuk mendapatkan range bin dan jumlah frekuensi sebagai berikut:

In [ ]:
df_can['2013'].head()

In [ ]:
#np.histogram returns 2 values
count, bin_edges = np.histogram(df_can['2013'])

print(count) #frequency count
print(bin_edges) # bin ranges, default = 10 bins

Secara default, metode histrogram memecah dataset menjadi 10 bin. Gambar di bawah ini merangkum kisaran bin dan distribusi frekuensi imigrasi di tahun 2013. Dapat kita lihat di tahun 2013:

178 negara menyumbang antara 0 hingga 3412,9 imigran
11 negara menyumbang antara 3412,9 hingga 6825,8 imigran
1 negara menyumbang antara 6285,8 hingga 10238,7 imigran, dan seterusnya..
Kita dapat dengan mudah membuat grafik distribusi ini dengan menyimpan kind=hist ke plot().

In [ ]:
df_can['2013'].plot(kind='hist', figsize=(8,5))
plt.title('Histogram of immigration from 195 countries in 2013')
plt.ylabel('Number of Countries')
plt.xlabel('Number of immigrant')
plt.show()

Pada plot di atas, sumbu x menunjukkan kisaran populasi imigran dalam interval 3412,9. Sumbu y mewakili jumlah negara yang berkontribusi pada populasi tersebut.

Perhatikan bahwa label sumbu x tidak cocok dengan ukuran bin. Hal ini dapat diperbaiki dengan memasukkan  xticks yang berisi daftar ukuran bin, sebagai berikut:


In [ ]:
import numpy as np

In [ ]:
count, bin_edges = np.histogram(df_can['2013'])

df_can['2013'].plot(kind='hist', figsize=(8, 5), xticks = bin_edges)

plt.title('Histogram of immigration from 195 countries in 2013')
plt.ylabel('Number of Countries')
plt.xlabel('Number of immigrant')
plt.show()

Side Note: Kita bisa menggunakan df_can['2013'].plot.hist(), sebagai gantinya. Faktanya, sepanjang sesi ini, menggunakan some_data.plot(kind='type_plot', ...) sama dengan some_data.plot.type_plot(...). Artinya, meneruskan tipe plot sebagai argumen atau metode akan berperilaku sama.

Lihat dokumentasi pandas untuk info selengkapnya http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.plot.html.

Kita juga dapat memplot beberapa histogram pada plot yang sama. Sebagai contoh, mari kita coba menjawab pertanyaan-pertanyaan berikut ini dengan menggunakan histogram.

Pertanyaan: Bagaimana distribusi imigrasi untuk Denmark, Norwegia, dan Swedia untuk tahun 1980 - 2013?

In [ ]:
df_can.loc[['Denmark','Norway','Sweden'], years]

In [ ]:
# Generate histogram
df_can.loc[['Denmark', 'Norway', 'Sweden'],years].plot.hist()

Kelihatannya tidak benar!
Splusinya sering kali terletak pada bagaimana kumpulan data yang mendasarinya di susun

Alih-alih memplot distribusi frekuensi papulasi untuk 3 negara, pandas malah memplot distribusi frekuensi untuk years

Kita akan perbaiki dengan terlebih dahulu mengubah ururtan dataset dan kemudian memplot seperti dibawah ini


In [ ]:
#trasnform dataframe
df_t = df_can.loc[['Denmark', 'Norway','Sweden'],years].transpose()
df_t.head()

In [ ]:
# Genearate histogram]
df_t.plot(kind='hist', figsize=(10,6))

plt.title('Histogram of immigration from 195 countries in 2013')
plt.ylabel('Number of Countries')
plt.xlabel('Number of immigrant')
plt.show()

Mari buat beberapa modifikasi untuk meningkatkan impact dan estetika dari plot sebelumnya:

tingkatkan ukuran bin menjadi 15 dengan meneruskan parameter bins
atur transparansi menjadi 60% dengan meneruskan parameter alfa
beri label sumbu x dengan melewatkan parameter label-x
ubah warna plot dengan memasukkan parameter warna


In [ ]:
df_t.info()

In [ ]:
count, bin_edges = np.histogram(df_t, 15)
print(count)
print(bin_edges)

In [ ]:
df_t.plot(kind ='hist', 
          figsize=(10, 6),
          bins=15,
          alpha=0.6,
          xticks=bin_edges,
          color=['coral', 'darkslateblue', 'mediumseagreen']
         )

plt.title('Histogram of Immigration from Denmark, Norway, and Sweden from 1980 - 2013')
plt.ylabel('Number of Years')
plt.xlabel('Number of Immigrants')

plt.show()

In [ ]:
import matplotlib
for name, hex in matplotlib.colors.cnames.items():
    print(name, hex)

In [ ]:
count, bin_edges = np.histogram(df_t, 15)
xmin = bin_edges[0] - 10   #  first bin value is 31.0, adding buffer of 10 for aesthetic purposes 
xmax = bin_edges[-1] + 10  #  last bin value is 308.0, adding buffer of 10 for aesthetic purposes

# stacked Histogram
df_t.plot(kind='hist',
          figsize=(10, 6), 
          bins=15,
          xticks=bin_edges,
          color=['coral', 'darkslateblue', 'mediumseagreen'],
          stacked=True,
          xlim=(xmin, xmax)
         )

plt.title('Histogram of Immigration from Denmark, Norway, and Sweden from 1980 - 2013')
plt.ylabel('Number of Years')
plt.xlabel('Number of Immigrants') 

plt.show()


### Bar Charts (Dataframe)

Plot batang/bar plot adalah cara merepresentasikan data di mana panjang batang merepresentasikan besaran / ukuran fitur / variabel. Grafik batang biasanya mewakili variabel numerik dan kategori yang dikelompokkan dalam interval.

Untuk membuat plot batang, kita dapat passing salah satu dari dua argumen melalui parameter kind di plot():

kind=bar creates a vertical bar plot
kind=barh creates a horizontal bar plot
Vertical bar plot

Dalam vertical bar graphs, sumbu x digunakan untuk labelling, dan panjang batang pada sumbu y sesuai dengan besaran variabel yang diukur. vertical bar graphs sangat berguna dalam menganalisis data deret waktu. Salah satu kelemahannya adalah kekurangan ruang untuk pelabelan teks di kaki setiap batang.

Let's start off by analyzing the effect of Iceland's Financial Crisis:

Krisis Keuangan Islandia 2008 - 2011 adalah peristiwa ekonomi dan politik utama di Islandia. Dibandingkan dengan peristiwa ekonominya, keruntuhan perbankan sistemik Islandia adalah yang terbesar dialami oleh negara mana pun dalam sejarah ekonomi. Krisis tersebut menyebabkan depresi ekonomi yang parah pada tahun 2008 - 2011 dan kerusuhan politik yang signifikan

Pertanyaan: Mari kita bandingkan jumlah imigran Islandia (negara = 'Islandia') ke Kanada dari tahun 1980 hingga 2013.

In [ ]:
# step 1: get the data
df_iceland = df_can.loc['Iceland', years]
df_iceland.head()

In [ ]:
# step 2: plot data
df_iceland.plot(kind='bar', figsize=(10, 6))

plt.xlabel('Year') # add to x-label to the plot
plt.ylabel('Number of immigrants') # add y-label to the plot
plt.title('Icelandic immigrants to Canada from 1980 to 2013') # add title to the plot

plt.show()

Plot di atas menunjukkan jumlah total imigran yang dikelompokkan setiap tahun. Kita bisa melihat dengan jelas dampak dari krisis keuangan; jumlah imigran ke Kanada mulai meningkat pesat setelah 2008.

Mari kita beri anotasi pada plot menggunakan metode annotate pada scripting layer atau antarmuka pyplot. Kita akan passing parameter berikut:

s: str, teks anotasi.
xy: Tuple yang menentukan titik (x,y) untuk diberi anotasi (dalam hal ini, titik akhir panah).
xytext: Tuple yang menentukan titik (x,y) untuk menempatkan teks (dalam hal ini, titik awal panah).
xycoords: Sistem koordinat yang diberikan xy di - 'data' menggunakan sistem koordinat objek yang dianotasi (default).
arrowprops: Mengambil kamus properti untuk menggambar panah:
arrowstyle: Menentukan gaya panah, '->' adalah panah standar.
connectionstyle: Menentukan jenis koneksi. arc3 adalah garis lurus.
color: Menentukan warna arror.
lw: Menentukan lebar garis.
Mari kita juga memberi anotasi pada teks untuk berada di atas panah. Kita akan menambahkan parameter tambahan berikut:

rotasi: sudut rotasi teks dalam derajat (berlawanan arah jarum jam)
va: perataan vertikal teks ['center' | 'top' | 'bottom' | 'baseline']
ha: perataan horizontal teks ['center' | 'right' | 'left']

In [ ]:
df_iceland.plot(kind='bar', figsize=(10, 6), rot=90) 

plt.xlabel('Year')
plt.ylabel('Number of Immigrants')
plt.title('Icelandic Immigrants to Canada from 1980 to 2013')

# Annotate arrow
plt.annotate('',                      # s: str. will leave it blank for no text
             xy=(32, 70),             # place head of the arrow at point (year 2012 , pop 70)
             xytext=(28, 20),         # place base of the arrow at point (year 2008 , pop 20)
             xycoords='data',         # will use the coordinate system of the object being annotated 
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='blue', lw=2)
            )

# Annotate Text
plt.annotate('2008 - 2011 Financial Crisis', # text to display
             xy=(28, 30),                    # start the text at at point (year 2008 , pop 30)
             rotation=72.5,                  # based on trial and error to match the arrow
             va='bottom',                    # want the text to be vertically 'bottom' aligned
             ha='left',                      # want the text to be horizontally 'left' algned.
            )

plt.show()

Horizontal Bar Plot

Terkadang lebih praktis untuk merepresentasikan data secara horizontal, terutama jika kita membutuhkan lebih banyak ruang untuk memberi label pada bar plot. Dalam horizontal bar graphs, sumbu y digunakan untuk pelabelan, dan panjang batang pada sumbu x sesuai dengan besaran variabel yang diukur. Seperti yang akan kita lihat, ada lebih banyak ruang pada sumbu y untuk memberi label variabel kategoris.

Pertanyaan: Dengan menggunakan lapisan skrip dan kumpulan data df_can, buat plot batang horizontal yang menunjukkan jumlah total imigran ke Kanada dari 15 negara teratas, untuk periode 1980 - 2013. Beri label setiap negara dengan jumlah total imigran.

Step 1: Dapatkan data yang berkaitan dengan 15 negara teratas.

In [ ]:
df_can.sort_values(by='Total', ascending=True, inplace=True)
df_top15 = df_can['Total'].tail(15)
df_top15

Step 2: Plot data:

Gunakan kind='barh' untuk membuat diagram batang dengan batang horizontal.
Pastikan untuk memilih ukuran plot yang baik dan memberi label pada kapak Anda dan memberi judul plot.
Ulangi negara dan beri anotasi pada populasi imigran menggunakan fungsi anotasi dari antarmuka skrip.

In [ ]:
### type your answer here

df_top15.plot(kind='barh', figsize=(12, 12), color='steelblue')
plt.xlabel('Number of Immigrants')
plt.title('Top 15 Conuntries Contributing to the Immigration to Canada between 1980 - 2013')
# for index, value in enumerate(df_top15): 
#     label = format(int(value), ',') # format int with commas
#     # place text at the end of bar (subtracting 47000 from x, and 0.1 from y to make it fit within the bar)
#     plt.annotate(label, xy=(value - 47000, index - 0.10), color='white')
plt.show()

### Pie Charts

Pie chart adalah grafik circualr yang menampilkan proporsi numerik dengan membagi lingkaran (atau pie) menjadi irisan proporsional. Kalian kemungkinan besar sudah terbiasa dengan pie chart karena diagram ini banyak digunakan dalam bisnis dan media. Kita dapat membuat pie chart di Matplotlib dengan memasukkan kind=pie.

Mari gunakan pie chart untuk menjelajahi proporsi (persentase) imigran baru yang dikelompokkan berdasarkan benua untuk seluruh periode waktu dari 1980 hingga 2013.

Step 1: Gather data.

Kita akan menggunakan metode pandas groupby untuk meringkas data imigrasi berdasarkan Continent. Proses umum groupby melibatkan langkah-langkah berikut:

Split: Memisahkan data menjadi beberapa kelompok berdasarkan beberapa kriteria.
Terapkan: Menerapkan fungsi ke setiap grup secara independen: .sum() .count() .mean() .std() .aggregate() .apply() .etc..
Combine: Menggabungkan hasil ke dalam struktur data.

In [ ]:
# group countries by continents and apply sum() function 
df_continents = df_can.groupby('Continent', axis=0).sum()
df_continents

In [ ]:
# note: the output of the groupby method is a `groupby' object. 
# we can not use it further until we apply a function (eg .sum())
print(type(df_can.groupby('Continent', axis=0)))

df_continents.head()

Step 2: Plot datanya. Kita akan passing kind = 'pie', bersama dengan parameter tambahan berikut:

autopct - adalah string atau fungsi yang digunakan untuk memberi label irisan dengan nilai numeriknya. Label akan ditempatkan di dalam irisan. Jika ini adalah format string, labelnya akan menjadi fmt%pct.
startangle - memutar awal diagram lingkaran dengan derajat sudut berlawanan arah jarum jam dari sumbu x.
shadow - Menggambar bayangan di bawah pai (untuk memberikan nuansa 3D).

In [ ]:
# autopct create %, start angle represent starting point
df_continents['Total'].plot(kind='pie',
                            figsize=(5, 6),
                            autopct='%1.1f%%', # add in percentages
                            startangle=90,     # start angle 90° (Africa)
                            shadow=True,       # add shadow      
                            )

plt.title('Immigration to Canada by Continent [1980 - 2013]')
plt.axis('equal') # Sets the pie chart to look like a circle.

plt.show()

Visual di atas tidak begitu jelas, angka dan teks tumpang tindih dalam beberapa kasus. Mari buat beberapa modifikasi untuk meningkatkan visual:

Hapus label teks pada diagram lingkaran dengan meneruskan legenda dan menambahkannya sebagai legenda terpisah menggunakan plt.legend().

dimajukan persentase untuk di tepat di luar diagram lingkaran dengan memasukkan parameter pct distance.

Lewati satu set warna khusus untuk benua dengan meneruskan parameter warna.

Explode diagram lingkaran untuk menekankan tiga benua terendah (Afrika, Amerika Utara, dan Amerika Latin dan Karibia) dengan memasukkan parameter meledak.

In [ ]:
colors_list = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen', 'pink']
explode_list = [0.1, 0, 0, 0, 0.1, 0.1] # ratio for each continent with which to offset each wedge.

df_continents['Total'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,         # turn off labels on pie chart
                            pctdistance=1.12,    # the ratio between the center of each pie slice and the start of the text generated by autopct 
                            colors=colors_list,  # add custom colors
                            explode=explode_list # 'explode' lowest 3 continents
                            )

# scale the title up by 12% to match pctdistance
plt.title('Immigration to Canada by Continent [1980 - 2013]', y=1.12) 

plt.axis('equal') 

# add legend
plt.legend(labels=df_continents.index, loc='upper left') 

plt.show()


Pertanyaan: Dengan menggunakan diagram lingkaran, telusuri proporsi (persentase) pendatang baru yang dikelompokkan berdasarkan benua pada tahun 2013.

Catatan: Anda mungkin perlu bermain dengan nilai explode untuk memperbaiki nilai irisan yang tumpang tindih.

In [ ]:
### type your answer here
colors_list = ['gold', 'yellowgreen', 'lightcoral', 'lightskyblue', 'lightgreen', 'pink']
explode_list = [0.1, 0, 0, 0, 0.05, 0.15] # ratio for each continent with which to offset each wedge.

df_continents['2013'].plot(kind='pie',
                            figsize=(15, 6),
                            autopct='%1.1f%%', 
                            startangle=90,    
                            shadow=True,       
                            labels=None,         # turn off labels on pie chart
                            pctdistance=1.12,    # the ratio between the center of each pie slice and the start of the text generated by autopct 
                            colors=colors_list,  # add custom colors
                            explode=explode_list # 'explode' lowest 3 continents
                            )

# scale the title up by 12% to match pctdistance
plt.title('Immigration to Canada by Continent Year 2013', y=1.12) 

plt.axis('equal') 

# add legend
plt.legend(labels=df_continents.index, loc='upper left') 

plt.show()

### Box Plots
Box plot adalah cara untuk secara statistik merepresentasikan distribusi data melalui lima dimensi utama:

Minimum: Smallest number in the dataset.

First quartile: Middle number between the minimum and the median.

Second quartile (Median): Middle number of the (sorted) dataset.

Third quartile: Middle number between median and maximum.

Maximum: Highest number in the dataset.

<img src="https://i.ibb.co/Sn9GCpL/img2.png" width="350" align="center">

Untuk membuat box plot, kita dapat menggunakan metode kind=box pada plot yang dipanggil pada serie pandas atau dataframe.

Mari kita plot box plot untuk para imigran Jepang antara 1980 - 2013.

Step 1: Dapatkan datasetnya. Meskipun kita mengekstrak data hanya untuk satu negara, kita akan menyimpannya sebagai dataframe. Ini akan membantu kita memanggil metode dataframe.describe() untuk melihat persentil.

In [ ]:
# to get a dataframe, place extra square brackets around 'Japan'.
df_japan = df_can.loc[['Japan'], years].transpose()
df_japan.head()

Step 2: Plot by Passing in kind='box

In [ ]:
df_japan.plot(kind='box', figsize=(8, 6))

plt.title('Box plot of Japanese Immigrants from 1980 - 2013')
plt.ylabel('Number of Immigrants')

plt.show()

Kami dapat segera membuat beberapa observasi penting dari plot di atas:


Jumlah minimum imigran sekitar 200 (min), 
jumlah maksimum sekitar 1300 (max), 
dan jumlah rata-rata imigran sekitar 900 (median).

25%  untuk periode 1980 - 2013 memiliki jumlah imigran tahunan ~500 atau kurang (Kuartil pertama).
75%  untuk periode 1980 - 2013 memiliki jumlah imigran tahunan ~1100 atau kurang (kuartil ketiga).

Kita bisa melihat angka aktual dengan memanggil metode describe() pada dataframe.

In [ ]:
df_japan.describe()

Salah satu manfaat utama dari box plots adalah membandingkan distribusi beberapa kumpulan data. 

Di salah satu plot sebelumnya, kita mengamati bahwa China dan India memiliki tren imigrasi yang sangat mirip. 

Mari kita analisis kedua negara ini lebih jauh menggunakan box plot.

Pertanyaan: Bandingkan distribusi jumlah pendatang baru dari India dan China periode 1980 - 2013.

Langkah 1: Dapatkan dataset untuk China dan India dan panggil dataframe df_CI.

In [ ]:
### type your answer here

df_CI = df_can.loc[(['China', 'India']), years].transpose()
df_CI.head()

In [ ]:
df_CI.describe()

In [ ]:
### type your answer here
df_CI.plot(kind='box', figsize=(8, 6))

plt.title('Box plot of China and India Immigrants from 1980 - 2013')
plt.ylabel('Number of Immigrants')

plt.show()

Kita dapat mengamati bahwa, 

meskipun kedua negara memiliki populasi imigran rata-rata yang sama (~ 20.000), 

range populasi imigran China lebih tersebar daripada India. 

Populasi maksimum dari India untuk setiap tahun (36.210) sekitar 15% lebih rendah dari populasi maksimum dari China (42.584).

Jika Kalian lebih suka membuat box plot horizontal, 

kalian dapat meneruskan parameter vert dalam fungsi plot dan menetapkannya ke False. 

Kalian juga dapat menentukan warna yang berbeda jika Kalian tidak suka warna default.

In [ ]:
# horizontal box plots
df_CI.plot(kind='box', figsize=(10, 7), color='blue', vert=False)

plt.title('Box plots of Immigrants from China and India (1980 - 2013)')
plt.xlabel('Number of Immigrants')

plt.show()

Subplots

Seringkali kita mungkin ingin memplot beberapa plot dalam figure yang sama. Misalnya, kita ingin melakukan perbandingan box plot secara berdampingan dengan line plot imigrasi China dan India.

Untuk memvisualisasikan beberapa plot bersama-sama, kita dapat membuat figure (overall canvas) dan membaginya menjadi subplots, masing-masing berisi plot. Dengan subplots, kita biasanya bekerja dengan artist layer daripada scripting layer.

Sintaks secara umum adalah:
```
fig = plt.figure() # create figure
ax = fig.add_subplot(nrows, ncols, plot_number) # create subplots

Dimana

* nrows and ```ncols``` are used to notionally split the figure into (nrows * ncols) sub-axes,
* ```plot_number``` is used to identify the particular subplot that this function is to create within the notional grid. plot_number starts at 1, increments across rows first and has a maximum of nrows * ncols as shown below.

<img src="https://i.ibb.co/kDCDyqv/img3.png" width="500" align="center">

Kita kemudian dapat menentukan subplot mana yang akan ditempatkan setiap plot dengan meneruskan parameter ax di plot() sebagai berikut

In [ ]:
fig = plt.figure() # create figure

ax0 = fig.add_subplot(1, 2, 1) # add subplot 1 (1 row, 2 columns, first plot)
ax1 = fig.add_subplot(122) # add subplot 2 (1 row, 2 columns, second plot). See tip below**

# Subplot 1: Box plot
df_CI.plot(kind='box', color='blue', vert=False, figsize=(20, 6), ax=ax0) # add to subplot 1
ax0.set_title('Box Plots of Immigrants from China and India (1980 - 2013)')
ax0.set_xlabel('Number of Immigrants')
ax0.set_ylabel('Countries')

# Subplot 2: Line plot
df_CI.plot(kind='line', figsize=(20, 6), ax=ax1) # add to subplot 2
ax1.set_title ('Line Plots of Immigrants from China and India (1980 - 2013)')
ax1.set_ylabel('Number of Immigrants')
ax1.set_xlabel('Years')

plt.show()


** * Tip regarding subplot convention **

Dalam kasus ketika nrows, ncols, dan plot_number semuanya kurang dari 10, ada kemudahan sedemikian rupa sehingga angka 3 digit dapat diberikan, di mana ratusan mewakili nrows, puluhan mewakili ncols dan unit mewakili plot_number. Misalnya,

   subplot(211) == subplot(2, 1, 1) #211 sama dengan 2, 1, 1


menghasilkan sub-sumbu dalam gambar yang mewakili plot teratas (yaitu yang pertama) dalam 2 baris kali 1 kolom.

Mari kita coba sesuatu yang lebih advanced.

Sebelumnya kita mengidentifikasi 15 negara teratas berdasarkan total imigrasi dari 1980 - 2013.

Question: Create a box plot to visualize the distribution of the top 15 countries (based on total immigration) grouped by the decades 1980s, 1990s, and 2000s.

Step 1: Dapatkan datasetnya. Dapatkan 15 negara teratas berdasarkan total populasi imigran. Beri nama dataframe df_top15.



In [ ]:
df_top15 = df_can.sort_values(['Total'], ascending=False, axis=0).head(15)
df_top15

Step 2: Buat dataframe baru yang berisi agregat untuk setiap dekade. Salah satu cara untuk melakukannya:

Buat daftar semua tahun dalam dekade 80-an, 90-an, dan 00-an.
Iris kerangka data asli df_can untuk membuat seri untuk setiap dekade dan jumlahkan sepanjang tahun untuk setiap negara.
Menggabungkan tiga seri menjadi bingkai data baru. Panggil kerangka data Anda new_df.

In [ ]:
years_80s = list(map(str, range(1980, 1990))) 
years_90s = list(map(str, range(1990, 2000))) 
years_00s = list(map(str, range(2000, 2010))) 

df_80s = df_top15.loc[:, years_80s].sum(axis=1) 
df_90s = df_top15.loc[:, years_90s].sum(axis=1) 
df_00s = df_top15.loc[:, years_00s].sum(axis=1)

new_df = pd.DataFrame({'1980s': df_80s, '1990s': df_90s, '2000s':df_00s}) 
new_df.head()

In [ ]:
new_df.describe()

In [ ]:
new_df.plot(kind='box', figsize=(10, 6))
plt.title('Immigration from top 15 countries for decades 80s, 90s and 2000s')
plt.show()

Perhatikan bagaimana box plot berbeda dari summary table yang dibuat. Box plot memindai data dan mengidentifikasi outliers. Untuk menjadi outliers, nilai datanya harus:

lebih besar dari Q3 dengan setidaknya 1,5 kali rentang interkuartil (IQR), atau,
lebih kecil dari Q1 setidaknya 1,5 kali IQR.

Mari kita lihat dekade 2000-an sebagai contoh:

Q1 (25%) = 36,101.5
Q3 (75%) = 105,505.5
IQR = Q3 - Q1 = 69,404

Menggunakan definisi outliers, nilai apa pun yang lebih besar dari Q3 sebesar 1,5 kali IQR akan ditandai sebagai outliers.

Outlier > 105,505.5 + (1.5 * 69,404)
Outlier > 209,611.5

In [ ]:
# let's check how many entries fall above the outlier threshold 
new_df[new_df['2000s']> 209611.5]

China dan India sama-sama dianggap sebagai outliers karena populasi mereka selama dekade tersebut melebihi 209.611,5.

Box plot adalah advanced visualizaiton tool, dan ada banyak opsi dan penyesuaian yang melebihi cakupan section ini. Silakan merujuk ke Matplotlib documentation pada box plot untuk informasi lebih lanjut.

### Scatter Plots
Scatter plot (2D) adalah metode yang berguna untuk membandingkan variabel satu sama lain. Scatter  plot terlihat mirip dengan line plot karena keduanya memetakan variabel independen dan dependen pada grafik 2D. Sementara datapoints  dihubungkan bersama oleh garis dalam line plot , datapoints tidak terhubung dalam scatter plot. Data dalam scatter plot dianggap mengekspresikan suatu tren. Dengan analisis lebih lanjut menggunakan tools seperti regresi, kita dapat menghitung hubungan ini secara matematis dan menggunakannya untuk memprediksi tren di luar kumpulan data.

Mari kita mulai dengan menjelajahi yang berikut ini:

Dengan menggunakan scatter plot, mari kita visualisasikan tren imigran total ke Kanada (semua negara digabungkan) untuk tahun 1980 - 2013.

Step 1: Dapatkan datasetnya. Karena kita menggunakan hubungan antara years dan total population, kita akan mengubah years menjadi tipe int .

In [ ]:
# we can use the sum() method to get the total population per year
df_tot = pd.DataFrame(df_can[years].sum(axis=0))

# change the years to type int (useful for regression later on)
df_tot.index = map(int, df_tot.index)

# reset the index to put in back in as a column in the df_tot dataframe
df_tot.reset_index(inplace = True)

# rename columns
df_tot.columns = ['year', 'total']

# view the final dataframe
df_tot.head()

Step 2: Plot datanya. Di Matplotlib, kita dapat membuat kumpulan scatter plot dengan meneruskan kind='scatter' sebagai argumen plot. Kita juga perlu memasukkan kata kunci x dan y untuk menentukan kolom yang berada pada sumbu x dan y.



In [ ]:
df_tot.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')

plt.title('Total Immigration to Canada from 1980 - 2013')
plt.xlabel('Year')
plt.ylabel('Number of Immigrants')

plt.show()

Notice how the scatter plot does not connect the datapoints together. We can clearly observe an upward trend in the data: as the years go by, the total number of immigrants increases. We can mathematically analyze this upward trend using a regression line (line of best fit).

Perhatikan bagaimana scatter plot tidak menghubungkan antar datapoints. Kita dapat dengan jelas mengamati tren peningkatan dalam data: seiring berlalunya waktu, jumlah total imigran meningkat. Kita dapat menganalisis tren naik ini secara matematis menggunakan regression line (line of best fit).

Question: Create a scatter plot of the total immigration from Denmark, Norway, and Sweden to Canada from 1980 to 2013?

Step 1: Get the data:

Create a dataframe the consists of the numbers associated with Denmark, Norway, and Sweden only. Name it df_countries.
Sum the immigration numbers across all three countries for each year and turn the result into a dataframe. Name this new dataframe df_total.
Reset the index in place.
Rename the columns to year and total.
Display the resulting dataframe.

In [ ]:
df_countries = df_can.loc[['Denmark', 'Norway', 'Sweden'], years].transpose()
df_total = pd.DataFrame(df_countries.sum(axis=1))
df_total.reset_index(inplace=True)
df_total.columns = ['year', 'total']
df_total['year'] = df_total['year'].astype(int)
df_total.head()

Step 2: Generate the scatter plot by plotting the total versus year in df_total.

In [ ]:
df_total.plot(kind='scatter', x='year', y='total', figsize=(10, 6), color='darkblue')
plt.title('Immigration from Denmark, Norway, and Sweden to Canada from 1980 - 2013')
plt.xlabel('Year')
plt.ylabel('Number of Immigrants')
plt.show()

Bubble Plots (Optional)
Bubble plot adalah variasi dari scatter plot yang menampilkan tiga dimensi data (x, y, z). Titik data diganti dengan bubbles, dan ukuran bubbles ditentukan oleh variabel ketiga 'z', yang juga dikenal sebagai bobot. Di maplotlib, kita bisa mengirimkan array atau skalar ke kata kunci s to plot(), yang berisi bobot setiap titik.

Let's start by analyzing the effect of Argentina's great depression.

Argentina mengalami great depression dari tahun 1998 - 2002, yang menyebabkan meluasnya pengangguran, kerusuhan, jatuhnya pemerintahan, dan gagal bayar hutang luar negeri negara tersebut. Dalam hal pendapatan, lebih dari 50% orang Argentina miskin, dan tujuh dari sepuluh anak Argentina miskin pada saat krisis tahun 2002.

Let's analyze the effect of this crisis, and compare Argentina's immigration to that of it's neighbour 

Mari kita analisis dampak krisis ini, dan bandingkan imigrasi Argentina dengan tetangganya Brasil. Mari kita lakukan itu dengan menggunakan bubble plot imigrasi dari Brasil dan Argentina untuk tahun 1980 - 2013. Kita akan menetapkan bobot gelembung sebagai nilai populasi yang dinormalisasi untuk setiap tahun.

Step 1: Get the data for Brazil and Argentina. Like in the previous example, we will convert the Years to type int and bring it in the dataframe.

In [ ]:
df_can_t = df_can[years].transpose() # transposed dataframe

# cast the Years (the index) to type int
df_can_t.index = map(int, df_can_t.index)

# let's label the index. This will automatically be the column name when we reset the index
df_can_t.index.name = 'Year'

# reset index to bring the Year in as a column
df_can_t.reset_index(inplace=True)

# view the changes
df_can_t.head()

Step 2: Create the normalized weights.

Ada beberapa metode normalisasi dalam statistik, masing-masing dengan kegunaannya sendiri. Dalam kasus ini, kita akan menggunakan feature scaling untuk membawa semua nilai ke dalam kisaran [0,1]. Rumus umumnya adalah:

<img src="https://i.ibb.co/KFNP2jh/img5.png" width="300" align="center">



di mana X adalah nilai original, X' adalah nilai yang dinormalisasi. Rumusnya menetapkan nilai maksimal dalam kumpulan data ke 1, dan menetapkan nilai min ke 0. Titik data lainnya diskalakan ke nilai antara 0-1.

In [ ]:
# normalize Brazil data
norm_brazil = (df_can_t['Brazil'] - df_can_t['Brazil'].min()) / (df_can_t['Brazil'].max() - df_can_t['Brazil'].min())

# normalize Argentina data
norm_argentina = (df_can_t['Argentina'] - df_can_t['Argentina'].min()) / (df_can_t['Argentina'].max() - df_can_t['Argentina'].min())

Step 3: Plot the data.

Untuk memplot dua plot pencar yang berbeda dalam satu plot, kita dapat memasukkan sumbu satu plot ke plot lainnya dengan melewatkannya melalui parameter axe.
Kami juga akan meneruskan bobot menggunakan parameter s. Mengingat bahwa bobot yang dinormalisasi adalah antara 0-1, bobot tersebut tidak akan terlihat di plot. Oleh karena itu kami akan:
kalikan bobot dengan 2000 untuk meningkatkannya pada grafik, dan,
tambahkan 10 untuk mengkompensasi nilai min (yang memiliki bobot 0 dan karenanya skala dengan x2000).

In [ ]:
# Brazil
ax0 = df_can_t.plot(kind='scatter',
                    x='Year',
                    y='Brazil',
                    figsize=(14, 8),
                    alpha=0.5,                  # transparency
                    color='green',
                    s=norm_brazil * 2000 + 10,  # pass in weights 
                    xlim=(1975, 2015)
                   )

# Argentina
ax1 = df_can_t.plot(kind='scatter',
                    x='Year',
                    y='Argentina',
                    alpha=0.5,
                    color="blue",
                    s=norm_argentina * 2000 + 10,
                    ax = ax0
                   )

ax0.set_ylabel('Number of Immigrants')
ax0.set_title('Immigration from Brazil and Argentina from 1980 - 2013')
ax0.legend(['Brazil', 'Argentina'], loc='upper left', fontsize='x-large')

Ukuran bubble sesuai dengan besarnya populasi yang berimigrasi untuk tahun itu, dibandingkan dengan data 1980 - 2013. Semakin besar bubble, semakin banyak imigran di tahun tersebut.

Dari plot di atas, kita dapat melihat peningkatan terkait imigrasi dari Argentina selama great depression 1998 - 2002. Kita juga dapat mengamati lonjakan serupa sekitar tahun 1985 hingga 1993. Faktanya, Argentina pernah mengalami great depression dari tahun 1974 - 1990, sebelum dimulainya great depression tahun 1998 - 2002.

Pada note yang sama, Brazil menderita Samba Effect dimana mata uang Brazil turun hampir 35% pada tahun 1999. Ada ketakutan akan krisis keuangan Amerika Selatan karena banyak negara Amerika Selatan sangat bergantung pada ekspor industri dari Brazil. Pemerintah Brasil kemudian mengadopsi program penghematan, dan ekonomi perlahan pulih selama bertahun-tahun, yang berpuncak pada lonjakan pada tahun 2010. Data imigrasi mencerminkan peristiwa ini.

Pertanyaan: Sebelumnya di bagian ini, kami membuat plot kotak untuk membandingkan imigrasi dari Cina dan India ke Kanada. Buat plot gelembung imigrasi dari Cina dan India untuk memvisualisasikan perbedaan apa pun dengan waktu dari tahun 1980 hingga 2013. Anda dapat menggunakan df_can_t yang telah kami definisikan dan gunakan pada contoh sebelumnya.


Step 1: Normalize the data pertaining to China and India.

In [ ]:
### type your answer here
norm_china = (df_can_t['China'] - df_can_t['China'].min()) / (df_can_t['China'].max() - df_can_t['China'].min())
norm_india = (df_can_t['India'] - df_can_t['India'].min()) / (df_can_t['India'].max() - df_can_t['India'].min())


Step 2: Generate the bubble plots

In [ ]:
ax0 = df_can_t.plot(kind='scatter',
                    x='Year',
                    y='China',
                    figsize=(14, 8),
                    alpha=0.5,                  # transparency
                    color='green',
                    s=norm_china * 1300 + 10,  # pass in weights 
                    xlim=(1975, 2015)
                   )

ax1 = df_can_t.plot(kind='scatter',
                    x='Year',
                    y='India',
                    alpha=0.5,
                    color="blue",
                    s=norm_india * 1300 + 10,
                    ax = ax0
                   )

ax0.set_ylabel('Number of Immigrants')
ax0.set_title('Immigration from China and India from 1980 - 2013')
ax0.legend(['China', 'India'], loc='upper left', fontsize='x-large')

In [ ]:
df = pd.read_excel('https://query.data.world/s/jswyvlfesptdycikd37t5ahgahi64f')

In [ ]:
def_head()

In [ ]:
segments = list(df["Segment"])
print(segments[:5])

In [ ]:
rekap_segment = {} 
for s in segments:
    if s in rekap_segment:
        rekap_segment[s] +=1 
    else: 
        rekap_segment[s] = 1

rekap_segment 

In [ ]:
import matplotlib.pyplot as plt 

segment_type = rekap_segment.keys() 
print(segment_type)
jml_segment = rekap_segment.values()

In [ ]:
plt.pie(jml_segment, 
        autopct='%1.0f%%',
        labels=segment_type, 
        startangle=60)
plt.axis('equal')
plt.title("Superstore Segmentation Buyer")
plt.show()

In [ ]:
date_order = df['Order Date']
date_order.head()

In [ ]:
date_order_str = list(map(lambda x: str(x), date_order)) 
date_order_str[:5]

In [ ]:
date_order_2015 = list(filter(lambda x : str(x[:4]) ==  '2019', date_order_str))
date_order_2015[:5]

In [ ]:
rekap_order = {} 
for tgl in date_order_2015:
    if tgl[5:7] in rekap_order:
        rekap_order[tgl[5:7]] +=1
    else:
        rekap_order[tgl[5:7]] =1

print(rekap_order)

In [ ]:
bulan_order = list(rekap_order.keys()) 
bulan_order.sort()
print(bulan_order)

In [ ]:
jumlah_order = [] 

for bln in bulan_order:
    jumlah_order.append(rekap_order[bln])

print(jumlah_order)

In [ ]:
plt.bar(bulan_order,jumlah_order, color="blue")
plt.title("Jumlah Order di 2020")
plt.ylabel("Jumlah")
plt.xlabel("Bulan")
plt.show()